In [46]:
import PyPDF2
from pathlib import Path
import re

In [47]:
f = "./data/RC_Gr2_S1_2019-20.pdf"
pdf = PyPDF2.PdfFileReader(f, False)
breaks = []
last_record = {'id': None, 'pages': []}
for p in range(0, pdf.numPages):
    this_record = {'id': None, 'pages': []}
    text = pdf.getPage(p).extractText()
    match = re.search('student\W?id:\W?(\d{5,})', text, re.IGNORECASE)
    name = re.search('^student:\W(.*)$', text, re.IGNORECASE)
    if match:
        breaks.append(last_record)
        this_record['id'] = match.group(1)
        this_record['pages'].append(p)
        last_record = this_record
    else:
        last_record['pages'].append(p)
print(breaks)

[{'id': None, 'pages': []}, {'id': '504637', 'pages': [0, 1, 2, 3, 4, 5]}, {'id': '505342', 'pages': [6, 7, 8, 9, 10, 11]}, {'id': '504641', 'pages': [12, 13, 14, 15, 16, 17]}, {'id': '504645', 'pages': [18, 19, 20, 21, 22, 23]}, {'id': '503967', 'pages': [24, 25, 26, 27, 28, 29]}, {'id': '505523', 'pages': [30, 31, 32, 33, 34, 35]}, {'id': '504390', 'pages': [36, 37, 38, 39, 40, 41]}, {'id': '505505', 'pages': [42, 43, 44, 45, 46, 47]}, {'id': '505206', 'pages': [48, 49, 50, 51, 52, 53]}, {'id': '505467', 'pages': [54, 55, 56, 57, 58, 59]}, {'id': '503969', 'pages': [60, 61, 62, 63, 64, 65]}, {'id': '503970', 'pages': [66, 67, 68, 69, 70, 71]}, {'id': '504869', 'pages': [72, 73, 74, 75, 76, 77]}, {'id': '505345', 'pages': [78, 79, 80, 81, 82, 83]}, {'id': '504732', 'pages': [84, 85, 86, 87, 88, 89]}, {'id': '505233', 'pages': [90, 91, 92, 93, 94, 95]}, {'id': '504944', 'pages': [96, 97, 98, 99, 100, 101]}, {'id': '503971', 'pages': [102, 103, 104, 105, 106, 107]}, {'id': '504744', 'pa

In [48]:
page = pdf.getPage(1)

In [70]:
text = pdf.getPage(1).extractText()

In [71]:
text

'2019-2020  School YearYousif AlanaziReport Card Grade 2Page number: 2Counting: Knows number names and the count sequence. Compares numbers.Numbers in Base Ten: Uses place value understanding and properties of operations to addand subtract.Operations and Algebraic Thinking: Understands addition and subtraction.Measurement and Data: Understands measurement: length, time, and data.Geometry: Analyzes, compares, creates, and composes shapes.Mathematical Practices: Exhibits habits of mind of a productive mathematical thinker.S S S S NA S MATHSemester 1Semester 2Reading: Reads closely to determine what the text says explicitly and makes logicalinferences from it; cites specific textual evidence when writing or speaking to supportconclusions drawn from the text.Reading: Determines central ideas or themes of a text and analyzes theirdevelopment; summarizes the key supporting details and ideas.Reading: Analyzes how and why individuals, events, and ideas develop and interactover the course of a 

In [64]:
m = re.search('student:\s?(.*)', text, re.IGNORECASE)
text

'Student: Yousif Alanazi2019-2020  School YearHomeroom Teacher: Marantos, KimReport Card - Grade 2Rijksstraatweg 200 2241 BX Wassenaar The Netherlands Telephone 070 - 512 1060 Telefax 070 - 511 24 00 www.ash.nlStudentID:504637Date: January 28, 2020Elementary School Principal: Tim MessickKEYB = Beginning Student performance is beginning to progress toward grade level expectations. Student requires consistentsupport or additional time to demonstrate skills and understanding of concepts and/or procedures.D = Developing Student performance is developing steadily toward grade level expectations. Student exhibits someindependence and may require occasional support or additional time to demonstrate skills and understanding of concepts and/orprocedures.S = Secure Student performance is secure and meets grade level expectations. Student independently demonstrates skills and anunderstanding of concepts and/or procedures. NA = Not Assessed Standard was not assessed this semester.Regulates own emo